In [1]:
import boto3
import sagemaker

In [2]:
print(boto3.__version__)
print(sagemaker.__version__)

1.17.39
2.32.0


In [3]:
sm = boto3.client('sagemaker')

In [4]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
print(bucket)
print(role)

sagemaker-us-east-2-805291263703
arn:aws:iam::805291263703:role/service-role/AmazonSageMaker-ExecutionRole-20200826T145400


In [5]:
prefix = 'pytorch-dogscats'

In [6]:
%%sh -s "$bucket" "$prefix"
aws s3 ls s3://$1/$2/

2021-03-30 07:57:26   87502442 dogscats.tar.gz
2021-03-30 08:32:00   87502442 model.tar.gz


In [7]:
import time

model_name = prefix+'-'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print(model_name)

pytorch-dogscats-2021-03-31-02-15-43


In [8]:
response = sm.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    Containers=[
        {
            'Image': '763104351884.dkr.ecr.us-east-2.amazonaws.com/pytorch-inference:1.5-cpu-py3',
            'ModelDataUrl': 's3://{}/{}/dogscats.tar.gz'.format(bucket, prefix),
            'Mode': 'SingleModel',
            'Environment': {
                'SAGEMAKER_PROGRAM': 'inference.py',
                'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code',
                'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
                'SAGEMAKER_REGION': 'us-east-2',
                'MMS_DEFAULT_RESPONSE_TIMEOUT': '500'
            }
        }
    ]
)
print(response)

{'ModelArn': 'arn:aws:sagemaker:us-east-2:805291263703:model/pytorch-dogscats-2021-03-31-02-15-43', 'ResponseMetadata': {'RequestId': '4639a7e3-9edc-40c1-a0e1-5e09cad7ce05', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4639a7e3-9edc-40c1-a0e1-5e09cad7ce05', 'content-type': 'application/x-amz-json-1.1', 'content-length': '98', 'date': 'Wed, 31 Mar 2021 02:15:53 GMT'}, 'RetryAttempts': 0}}


In [9]:
epc_name = prefix+'-epc'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print(epc_name)

pytorch-dogscats-epc2021-03-31-02-15-57


In [10]:
response = sm.create_endpoint_config(
    EndpointConfigName=epc_name,
    ProductionVariants=[
        {
            'VariantName': 'variant-1',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.m5.large',
            'InitialVariantWeight': 1,
            'AcceleratorType': 'ml.eia1.medium',
        }
    ]
)
print(response)

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:805291263703:endpoint-config/pytorch-dogscats-epc2021-03-31-02-15-57', 'ResponseMetadata': {'RequestId': 'ecf9b44b-ad0d-4726-8aeb-0ac48ef08d62', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'ecf9b44b-ad0d-4726-8aeb-0ac48ef08d62', 'content-type': 'application/x-amz-json-1.1', 'content-length': '120', 'date': 'Wed, 31 Mar 2021 02:16:50 GMT'}, 'RetryAttempts': 0}}


In [11]:
ep_name=prefix+'-ep'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print(ep_name)

pytorch-dogscats-ep2021-03-31-02-16-54


In [12]:
response=sm.create_endpoint(
    EndpointName=ep_name,
    EndpointConfigName=epc_name
)
print(response)

ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateEndpoint operation: The account-level service limit 'ml.eia1.medium for endpoint usage' is 0 Accelerators, with current utilization of 0 Accelerators and a request delta of 1 Accelerators. Please contact AWS support to request an increase for this limit.

In [13]:
sm.describe_endpoint(EndpointName=ep_name)

{'EndpointName': 'pytorch-dogscats-ep2021-03-30-04-56-24',
 'EndpointArn': 'arn:aws:sagemaker:us-east-2:805291263703:endpoint/pytorch-dogscats-ep2021-03-30-04-56-24',
 'EndpointConfigName': 'pytorch-dogscats-epc2021-03-30-04-56-21',
 'EndpointStatus': 'Creating',
 'CreationTime': datetime.datetime(2021, 3, 30, 4, 56, 25, 855000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 3, 30, 4, 56, 25, 855000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '24306866-85cd-4e16-954c-a120fa94374d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '24306866-85cd-4e16-954c-a120fa94374d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '321',
   'date': 'Tue, 30 Mar 2021 04:56:27 GMT'},
  'RetryAttempts': 0}}

In [14]:
waiter = sm.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=ep_name)

# Get model inference

In [42]:
import cv2
import os
import numpy as np
from io import BytesIO
import torch

In [29]:
TEST_DIR = './test1'

In [30]:
test_imgs = os.listdir(TEST_DIR)

In [60]:
smrt = boto3.client('runtime.sagemaker')

In [86]:
for i in range(1000):
    im = cv2.imread(os.path.join(TEST_DIR, test_imgs[i]))
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = cv2.resize(im, (224,224))
    tensor_x = np.expand_dims(im, axis=0)
    tensor_x = tensor_x.transpose((0,3,1,2))
    buffer = BytesIO()
    np.save(buffer, tensor_x)
    
    response = smrt.invoke_endpoint(
        EndpointName=ep_name,
        Body=buffer.getvalue(),
        ContentType='application/x-npy'
    )

#     print(response['Body'].read())

# Delete endpoint, endpoint configuration and model

In [15]:
sm.delete_endpoint(EndpointName=ep_name)

{'ResponseMetadata': {'RequestId': 'b608a610-167c-4348-a632-be6a8fb66de5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b608a610-167c-4348-a632-be6a8fb66de5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 30 Mar 2021 05:06:15 GMT'},
  'RetryAttempts': 0}}

In [16]:
sm.delete_endpoint_config(EndpointConfigName=epc_name)

{'ResponseMetadata': {'RequestId': 'fcfbcb9e-694f-4711-9c50-e26e013b64d7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fcfbcb9e-694f-4711-9c50-e26e013b64d7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 30 Mar 2021 05:06:15 GMT'},
  'RetryAttempts': 0}}

In [17]:
sm.delete_model(ModelName=model_name)

{'ResponseMetadata': {'RequestId': '4cc3efef-0c3d-4eb0-b767-1c70e3448eec',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4cc3efef-0c3d-4eb0-b767-1c70e3448eec',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 30 Mar 2021 05:06:17 GMT'},
  'RetryAttempts': 0}}